In [2]:
%pylab inline
import keras
from keras.layers import Dense, Flatten, Input, Conv2D
from keras.models import Model

Populating the interactive namespace from numpy and matplotlib


Using Theano backend.
Using cuDNN version 5110 on context None
Mapped name None to device cuda: Tesla K20Xm (0000:20:00.0)


# Научимся делать из полносвязных слоёв свёрточные

Сделаем несколько простых моделей с которыми будем эксперементировать

In [3]:
inp = Input(shape=(5,5,3))

x = Flatten()(inp)
m0 = Model(inputs=inp, outputs=x) # Input + Flatten

x = Dense(2, activation=None)(x)  
m1 = Model(inputs=inp, outputs=x) # Input + Flatten + Dense

In [4]:
x = Conv2D(filters=2, kernel_size=(5,5))(inp)
m2 = Model(inputs=inp, outputs=x) # Input + Conv2D

In [6]:
print 'Dense params shape: ', m1.layers[-1].get_weights()[0].shape
print 'Conv2D params shape:', m2.layers[-1].get_weights()[0].shape

Dense params shape:  (75, 2)
Conv2D params shape: (5, 5, 3, 2)


In [7]:
# Сгенерируем конкретные веса для экспериментов
dense_weights_W = np.array(range(1, 5*5*3*2 + 1)).reshape(5*5*3, 2).astype('float32')
dense_weights_b = np.array([10000000, 20000000]).astype('float32')

In [8]:
# И конкретный пример
input_x = np.array(range(5*5*3, 0, -1)).reshape(1, 5,5,3).astype('float32')

Для начала убедимся, что слой Flatten работает предсказуемо

In [9]:
input_x.flatten()

array([ 75.,  74.,  73.,  72.,  71.,  70.,  69.,  68.,  67.,  66.,  65.,
        64.,  63.,  62.,  61.,  60.,  59.,  58.,  57.,  56.,  55.,  54.,
        53.,  52.,  51.,  50.,  49.,  48.,  47.,  46.,  45.,  44.,  43.,
        42.,  41.,  40.,  39.,  38.,  37.,  36.,  35.,  34.,  33.,  32.,
        31.,  30.,  29.,  28.,  27.,  26.,  25.,  24.,  23.,  22.,  21.,
        20.,  19.,  18.,  17.,  16.,  15.,  14.,  13.,  12.,  11.,  10.,
         9.,   8.,   7.,   6.,   5.,   4.,   3.,   2.,   1.], dtype=float32)

In [10]:
m0.predict(input_x)

array([[ 75.,  74.,  73.,  72.,  71.,  70.,  69.,  68.,  67.,  66.,  65.,
         64.,  63.,  62.,  61.,  60.,  59.,  58.,  57.,  56.,  55.,  54.,
         53.,  52.,  51.,  50.,  49.,  48.,  47.,  46.,  45.,  44.,  43.,
         42.,  41.,  40.,  39.,  38.,  37.,  36.,  35.,  34.,  33.,  32.,
         31.,  30.,  29.,  28.,  27.,  26.,  25.,  24.,  23.,  22.,  21.,
         20.,  19.,  18.,  17.,  16.,  15.,  14.,  13.,  12.,  11.,  10.,
          9.,   8.,   7.,   6.,   5.,   4.,   3.,   2.,   1.]], dtype=float32)

А теперь попробуем преобразовать веса Dense слоя в веса для Conv2D слоя так, чтобы результат выполнения слоёв был одинаковым.

In [11]:
# Задаём Dense слою наши веса
m1.layers[-1].set_weights([dense_weights_W, dense_weights_b])
m1.predict(input_x)

array([[ 10143450.,  20146300.]], dtype=float32)

In [14]:
# Попробуем преобразовать
conv_W = dense_weights_W.reshape(5,5,3,2)

In [15]:
m2.layers[-1].set_weights([conv_W, dense_weights_b])
m2.predict(input_x)

array([[[[ 10283850.,  20286700.]]]], dtype=float32)

In [16]:
# Попытка №2
conv_W = dense_weights_W.reshape(5,5,3,2) [::-1, ::-1, :, :]

In [17]:
m2.layers[-1].set_weights([conv_W, dense_weights_b])
m2.predict(input_x)

array([[[[ 10143450.,  20146300.]]]], dtype=float32)